# Training and Deploying a BlazingText model

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3
session = sagemaker.Session()
role = get_execution_role()
region_name = boto3.Session().region_name

In [ ]:
%store -r s3_bucket
%store -r prefix

In [ ]:
s3_train_data = 's3://{}/{}/input/{}'.format(
    s3_bucket, 
    prefix, 
    "synthetic.train.txt"
)
s3_validation_data = 's3://{}/{}/input/{}'.format(
    s3_bucket, 
    prefix, 
    "synthetic.validation.txt"
)
s3_output_location = 's3://{}/{}/output'.format(
    s3_bucket, 
    prefix
)

In [ ]:
from sagemaker.image_uris import retrieve 

container = retrieve(
    "blazingtext", 
    region_name, 
    "1"
)

In [ ]:
estimator = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.c4.xlarge',
    input_mode= 'File',
    output_path=s3_output_location,
    sagemaker_session=session
)

In [ ]:
estimator.set_hyperparameters(
    mode="supervised", 
    min_count=2
)

In [ ]:
from sagemaker.inputs import TrainingInput

train_data = TrainingInput(
    s3_train_data, 
    distribution='FullyReplicated',   
    content_type='text/plain', 
    s3_data_type='S3Prefix'
)

validation_data = TrainingInput(
    s3_validation_data, 
    distribution='FullyReplicated', 
    content_type='text/plain', 
    s3_data_type='S3Prefix'
)

In [ ]:
data_channels = {
    'train': train_data, 
    'validation': validation_data
}

In [ ]:
%%time
estimator.fit(
    inputs=data_channels, 
    logs=True
)

In [ ]:
endpoint = estimator.deploy(
    initial_instance_count = 1, 
    instance_type = 'ml.r5.large'
)

In [ ]:
sentences = [
    "that is bad", 
    "the apple tastes good", 
    "i would recommend it to my friends"
]

payload = {"instances" : sentences}

In [ ]:
from sagemaker.serializers import JSONSerializer

endpoint.serializer = JSONSerializer()
response = endpoint.predict(payload)
predictions = json.loads(response)

print(json.dumps(predictions, indent=2))

In [ ]:
tn = estimator.latest_training_job.name
training_job_name = tn
%store training_job_name

In [ ]:
# endpoint.delete_endpoint()